In [ ]:
def generate_description(table_name, columns, sample_data):
    all_descriptions = {}
    for column in columns:
        column_name = list(column.keys())[0]
        column_type = column[column_name]
        prompt = f"Table: {table_name}, Column: {column_name}, Type: {column_type}, Sample Data: {sample_data}"
        # Assuming an LLM agent function call here, replace with your actual LLM agent call
        description = call_llm_agent(prompt)  # Placeholder for your LLM agent function
        if table_name not in all_descriptions:
            all_descriptions[table_name] = {}
        all_descriptions[table_name][column_name] = description
    return all_descriptions

def call_llm_agent(prompt):
    # Placeholder for your LLM agent function that processes the prompt and returns a description
    # Replace this with your actual LLM agent implementation
    return f"Description for {prompt.split(',')[1].strip()} based on provided sample data."

# Example usage with the provided schema
schema = {
    "movie": {
        "columns": [
            {"available_globally": "NUMERIC"},
            {"id": "INTEGER"},
            {"runtime": "INTEGER"},
            {"locale": "TEXT"},
            {"original_title": "TEXT"},
            {"title": "TEXT"},
            {"release_date": "TEXT"},
            {"created_date": "TEXT"},
            {"modified_date": "TEXT"}
        ],
        "sample": [
            {"available_globally": 1, "id": 1, "runtime": 110, "locale": None, "original_title": None, "title": "Damsel", "release_date": "2024-03-08 13:30:00", "created_date": "2024-01-01 05:30:00", "modified_date": "2024-01-01 05:30:00"},
            {"available_globally": 1, "id": 2, "runtime": 107, "locale": None, "original_title": None, "title": "Lift", "release_date": "2024-01-12 13:30:00", "created_date": "2024-01-01 05:30:00", "modified_date": "2024-01-01 05:30:00"},
        ]
    }
}

# Process each table in the schema
all_descriptions = {}
for table_name, details in schema.items():
    all_descriptions[table_name] = generate_description(table_name, details["columns"], details["sample"])

# Save descriptions to a file (example with JSON format)
import json
with open(f"{table_name}_descriptions.json", "w") as f:
    json.dump(all_descriptions, f, indent=4)